In [ ]:
import sys, os
#sys.path.append(os.getcwd())
sys.path.append(os.path.abspath('..'))

from user_profile import load_file, get_project_paths

paths = get_project_paths()

# Load data
raw = paths['data_raw']
df1_indicator = load_file(os.path.join(raw, 'fusion_GLOBAL_DATAFLOW_UNICEF_1.0_.MNCH_ANC4+MNCH_SAB..csv'))
df2_track = load_file(os.path.join(raw, 'On-track and off-track countries.xlsx'), sheet_name='Sheet1', header_row=0)
df3_estimates = load_file(os.path.join(raw, 'WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT_REV1.xlsx'), sheet_name='Estimates', header_row=16)
df3_projections = load_file(os.path.join(raw, 'WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT_REV1.xlsx'), sheet_name='Projections', header_row=16)

# Extract ISO3
print("Extracting ISO3 codes and cleaning...")
df1_indicator['ISO3Code'] = df1_indicator['REF_AREA:Geographic area'].str.extract(r'^([A-Z]{3})')
df3_estimates.rename(columns={'ISO3 Alpha-code': 'ISO3Code'}, inplace=True)
df3_projections.rename(columns={'ISO3 Alpha-code': 'ISO3Code'}, inplace=True)

def clean_iso_column(df, col='ISO3Code'):
    df = df.copy()
    df[col] = df[col].astype(str).str.strip().str.upper()
    df = df[df[col].notna()]
    df.drop_duplicates(subset=[col], inplace=True)
    return df

df1_indicator = clean_iso_column(df1_indicator)
df2_track = clean_iso_column(df2_track)
df3_estimates = clean_iso_column(df3_estimates)
df3_projections = clean_iso_column(df3_projections)

# Filter by common ISO3 codes
codes_df1 = set(df1_indicator['ISO3Code'].dropna())
codes_df2 = set(df2_track['ISO3Code'].dropna())
codes_df3 = set(df3_estimates['ISO3Code'].dropna())
codes_df4 = set(df3_projections['ISO3Code'].dropna())
common_iso3 = codes_df1 & codes_df2 & codes_df3 & codes_df4

df1_common = df1_indicator[df1_indicator['ISO3Code'].isin(common_iso3)]
df2_common = df2_track[df2_track['ISO3Code'].isin(common_iso3)]
df3_common_estimates = df3_estimates[df3_estimates['ISO3Code'].isin(common_iso3)]
df3_common_projections = df3_projections[df3_projections['ISO3Code'].isin(common_iso3)]

# Save processed
processed = paths['data_processed']
df1_common.to_csv(os.path.join(processed, 'df1_common_indicator.csv'), index=False)
df2_common.to_csv(os.path.join(processed, 'df2_common_track.csv'), index=False)
df3_common_estimates.to_csv(os.path.join(processed, 'df3_common_estimates.csv'), index=False)
df3_common_projections.to_csv(os.path.join(processed, 'df3_common_projections.csv'), index=False)

print("✅ Data preparation completed. Processed files saved.")


ModuleNotFoundError: No module named 'user_profile'